In [1]:
import json
import hashlib
import numpy as np
import os
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

# Connecting to Google Drive

This section is unnecessary if running locally, but needed if running on Colab.  If you are going to bring in your data this way, be sure to create `/Senzing/data` on your Google Drive and put the `occrp_17k.csv` file there.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Senzing/data')

Mounted at /content/drive


In [3]:
df = pd.read_csv('occrp_17k.csv', dtype=str)
null_counts = df.isnull().sum()
print(null_counts)

payer_name                    0
payer_jurisdiction            0
payer_account               115
source_file                   0
amount_orig                   0
id                            0
beneficiary_type              0
beneficiary_core              0
amount_orig_currency          0
beneficiary_name              0
beneficiary_jurisdiction      0
investigation                 0
beneficiary_bank_country      4
beneficiary_name_norm         0
payer_core                    0
beneficiary_account           4
purpose                       0
date                          0
amount_usd                    0
amount_eur                    0
payer_type                    0
payer_name_norm               0
payer_bank_country          115
dtype: int64


In [4]:
df.replace(['NA', 'NULL', 'NUL', 'null'], np.nan, inplace=True)
df.fillna(value='UNKNOWN', inplace=True)
df.head()

,payer_name,payer_jurisdiction,payer_account,source_file,amount_orig,id,beneficiary_type,beneficiary_core,amount_orig_currency,beneficiary_name,beneficiary_jurisdiction,investigation,beneficiary_bank_country,beneficiary_name_norm,payer_core,beneficiary_account,purpose,date,amount_usd,amount_eur,payer_type,payer_name_norm,payer_bank_country
0,AZARBAYCAN METANOL KOMPANI MMC,AZ,33817018409333311204,pdf/LCM ALLIANCE Account statement 30.06.12-31...,535470.0,6049,Company,True,USD,LCM ALLIANCE LLP,GB,az,EE,LCM ALLIANCE LLP,False,EE27 3300 3335 0561 0002,1206295100052180 OCT4121800021 ADVANCE PAYM FO...,2012-06-30,535470,"$431,762.31",Company,AZARBAYCAN METANOL KOMPANI MMC,33
1,LCM ALLIANCE LLP,GB,EE27 3300 3335 0561 0002,pdf/LCM ALLIANCE Account statement 30.06.12-31...,-535000.0,6050,Company,False,USD,MOBILA LLP,GB,az,33,MOBILA LLP,True,333504500003,1207035026699176 INVOICE.No 6215 DD 25.05.2012,2012-07-03,535000,"$423,688.44",Company,LCM ALLIANCE LLP,EE
2,SKN ELECTRICAL SERVICES LIMITEDACCO,GB,20100374548222,pdf/METASTAR Account statement 30.06.12-31.12....,90535.19,10623,Company,True,USD,METASTAR INVEST LLP,GB,az,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,"1207065103089249 /FEE/USD4,81",2012-07-06,90536,"$71,698.53",Company,SKN ELECTRICAL SERVICES LIMITED,20
3,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-90520.0,15589,Company,False,USD,INMAXO CAPITAL CORP,VG,az,33,INMAXO CAPITAL CORP.,True,333455870002,1207095022358525 DOGOVOR ZAYMA,2012-07-09,90520,"$71,686.50",Company,METASTAR INVEST LLP,EE
4,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-60.0,10624,Company,False,USD,INMAXO CAPITAL CORP,VG,az,33,INMAXO CAPITAL CORP.,True,333455870002,1207135024578077 DOGOVOR ZAYMA,2012-07-13,60,$47.52,Company,METASTAR INVEST LLP,EE


In [5]:
df.shape

(16940, 23)

In [6]:
df.columns

Index(['payer_name', 'payer_jurisdiction', 'payer_account', 'source_file',
       'amount_orig', 'id', 'beneficiary_type', 'beneficiary_core',
       'amount_orig_currency', 'beneficiary_name', 'beneficiary_jurisdiction',
       'investigation', 'beneficiary_bank_country', 'beneficiary_name_norm',
       'payer_core', 'beneficiary_account', 'purpose', 'date', 'amount_usd',
       'amount_eur', 'payer_type', 'payer_name_norm', 'payer_bank_country'],
      dtype='object')

In [7]:
print(df['payer_type'].unique())
print(df['beneficiary_type'].unique())

['Company' 'Invalid' 'Person']
['Company' 'Person' 'Invalid']


# Separating into payers and beneficiaries and then further breaking into the 3 different entity types in this dataset

In [8]:
payer_columns = [col for col in df.columns if col.startswith('payer')]
beneficiary_columns = [col for col in df.columns if col.startswith('beneficiary')]

# Create payer DataFrames with only payer columns
payer_companies = df[df['payer_type'] == 'Company'][payer_columns].drop_duplicates()
payer_persons = df[df['payer_type'] == 'Person'][payer_columns].drop_duplicates()
payer_invalids = df[df['payer_type'] == 'Invalid'][payer_columns].drop_duplicates()

# Create beneficiary DataFrames with only beneficiary columns
beneficiary_companies = df[df['beneficiary_type'] == 'Company'][beneficiary_columns].drop_duplicates()
beneficiary_persons = df[df['beneficiary_type'] == 'Person'][beneficiary_columns].drop_duplicates()
beneficiary_invalids = df[df['beneficiary_type'] == 'Invalid'][beneficiary_columns].drop_duplicates()

payer_companies.shape, payer_persons.shape, payer_invalids.shape, beneficiary_companies.shape, beneficiary_persons.shape, beneficiary_invalids.shape

((442, 7), (10, 7), (22, 7), (3361, 7), (429, 7), (3, 7))

In [9]:
payer_companies.head()

,payer_name,payer_jurisdiction,payer_account,payer_core,payer_type,payer_name_norm,payer_bank_country
0,AZARBAYCAN METANOL KOMPANI MMC,AZ,33817018409333311204,False,Company,AZARBAYCAN METANOL KOMPANI MMC,33
1,LCM ALLIANCE LLP,GB,EE27 3300 3335 0561 0002,True,Company,LCM ALLIANCE LLP,EE
2,SKN ELECTRICAL SERVICES LIMITEDACCO,GB,20100374548222,False,Company,SKN ELECTRICAL SERVICES LIMITED,20
3,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,True,Company,METASTAR INVEST LLP,EE
5,INMAXO CAPITAL CORP.,VG,333455870002,False,Company,INMAXO CAPITAL CORP.,33


In [10]:
beneficiary_companies.head()

,beneficiary_type,beneficiary_core,beneficiary_name,beneficiary_jurisdiction,beneficiary_bank_country,beneficiary_name_norm,beneficiary_account
0,Company,True,LCM ALLIANCE LLP,GB,EE,LCM ALLIANCE LLP,EE27 3300 3335 0561 0002
1,Company,False,MOBILA LLP,GB,33,MOBILA LLP,333504500003
2,Company,True,METASTAR INVEST LLP,GB,EE,METASTAR INVEST LLP,EE77 3300 3334 8704 0004
3,Company,False,INMAXO CAPITAL CORP,VG,33,INMAXO CAPITAL CORP.,333455870002
7,Company,False,JETFIELD NETWORKS LIMITED,NZ,33,JETFIELD NETWORKS LIMITED,333488110001


# How the dataframe columns will get mapped into the Senzing format

In [11]:
payer_mapping = {
    'DATA_SOURCE': 'PAYERS',
    'RECORD_ID': 'payer_record_id',
    'RECORD_TYPE': None,  # To be set based on the DataFrame (COMPANY, PERSON, INVALID)
    'NAME_ORG': 'payer_name',
    'ADDR_COUNTRY': 'payer_jurisdiction',
    'ACCOUNT_NUMBER': 'payer_account',
    'ACCOUNT_DOMAIN': 'payer_bank_country'
}

beneficiary_mapping = {
    'DATA_SOURCE': 'BENEFICIARIES',
    'RECORD_ID': 'beneficiary_record_id',
    'RECORD_TYPE': None,  # To be set based on the DataFrame (COMPANY, PERSON, INVALID)
    'NAME_ORG': 'beneficiary_name',
    'ADDR_COUNTRY': 'beneficiary_jurisdiction',
    'ACCOUNT_NUMBER': 'beneficiary_account',
    'ACCOUNT_DOMAIN': 'beneficiary_bank_country'
}

# Function to do it for each dataframe

In [12]:
def generate_json(df, mapping, record_type):
    # Create a copy to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Set constant values
    df_copy['DATA_SOURCE'] = mapping['DATA_SOURCE']
    df_copy['RECORD_TYPE'] = record_type.upper()  # Ensure RECORD_TYPE is uppercase

    # Rename columns according to the mapping
    df_copy.rename(columns={
        mapping['NAME_ORG']: 'NAME_ORG',
        mapping['ADDR_COUNTRY']: 'ADDR_COUNTRY',
        mapping['ACCOUNT_NUMBER']: 'ACCOUNT_NUMBER',
        mapping['ACCOUNT_DOMAIN']: 'ACCOUNT_DOMAIN'
    }, inplace=True)

    # Select the required columns and create another copy to ensure no warnings
    columns_order = ['DATA_SOURCE', 'RECORD_TYPE', 'NAME_ORG',
                     'ADDR_COUNTRY', 'ACCOUNT_NUMBER', 'ACCOUNT_DOMAIN']
    df_json = df_copy[columns_order].copy()  # Explicitly copy the selected DataFrame

    # Ensure all columns are strings to prevent numeric conversion
    for col in columns_order:
        df_json[col] = df_json[col].astype(str)

    # Create the RECORD_ID column by hashing the concatenated values of the columns
    def generate_record_id(row):
        # Concatenate all values in the row into a single string
        concatenated_values = ''.join(row[col] for col in columns_order)
        # Generate the MD5 hash
        return hashlib.md5(concatenated_values.encode('utf-8')).hexdigest()

    # Apply the function to generate RECORD_ID for each row
    df_json['RECORD_ID'] = df_json.apply(generate_record_id, axis=1)

    # Convert DataFrame to JSON records
    records = df_json.to_dict(orient='records')

    return records


In [13]:
payer_companies_json = generate_json(payer_companies, payer_mapping, record_type='COMPANY')
payer_persons_json = generate_json(payer_persons, payer_mapping, record_type='PERSON')
payer_invalids_json = generate_json(payer_invalids, payer_mapping, record_type='INVALID')

In [14]:
payer_companies_json[0]

{'DATA_SOURCE': 'PAYERS',
 'RECORD_TYPE': 'COMPANY',
 'NAME_ORG': 'AZARBAYCAN METANOL KOMPANI MMC',
 'ADDR_COUNTRY': 'AZ',
 'ACCOUNT_NUMBER': '33817018409333311204',
 'ACCOUNT_DOMAIN': '33',
 'RECORD_ID': '991649265e1a7bd9a5fc4d174e8a4cd7'}

In [36]:
with open('payer_companies.json', 'w') as f:
    for record in payer_companies_json:
        json.dump(record, f)
        f.write('\n')  # Write each record on a new line

with open('payer_persons.json', 'w') as f:
    for record in payer_persons_json:
        json.dump(record, f)
        f.write('\n')

with open('payer_invalids.json', 'w') as f:
    for record in payer_invalids_json:
        json.dump(record, f)
        f.write('\n')

In [37]:
beneficiary_companies_json = generate_json(beneficiary_companies, beneficiary_mapping, record_type='COMPANY')
beneficiary_persons_json = generate_json(beneficiary_persons, beneficiary_mapping, record_type='PERSON')
beneficiary_invalids_json = generate_json(beneficiary_invalids, beneficiary_mapping, record_type='INVALID')

In [38]:
beneficiary_companies_json[0]

{'DATA_SOURCE': 'BENEFICIARIES',
 'RECORD_TYPE': 'COMPANY',
 'NAME_ORG': 'LCM ALLIANCE LLP',
 'ADDR_COUNTRY': 'GB',
 'ACCOUNT_NUMBER': 'EE27 3300 3335 0561 0002',
 'ACCOUNT_DOMAIN': 'EE',
 'RECORD_ID': 'c56afa69f8e535cfa43c99c74bb7fc82'}

In [39]:
with open('beneficiary_companies.json', 'w') as f:
    for record in beneficiary_companies_json:
        json.dump(record, f)
        f.write('\n')

with open('beneficiary_persons.json', 'w') as f:
    for record in beneficiary_persons_json:
        json.dump(record, f)
        f.write('\n')

with open('beneficiary_invalids.json', 'w') as f:
    for record in beneficiary_invalids_json:
        json.dump(record, f)
        f.write('\n')